In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

try:
    import seaborn as sns
except ImportError:
    print('Seabofn is not installed')
%matplotlib inline

from tyssue import Sheet, config
from tyssue import SheetGeometry as geom
from tyssue.core.generation import three_faces_sheet

from tyssue.draw.threejs_draw import view_3js
from IPython.display import display

# Grab the default settings for drawing
draw_specs = config.draw.sheet_spec()

### Create a sheet

In [2]:
datasets, specs = three_faces_sheet()
sheet = Sheet('3cells_2D', datasets, specs)
geom.update_all(sheet)

## Default 

In [3]:
renderer = view_3js(sheet)

display(renderer)

(<pythreejs.pythreejs.Renderer at 0x7f9be8a12c18>,
 <pythreejs.pythreejs.Line at 0x7f9be8a12630>)

In [18]:
from tyssue.io.hdf5 import load_datasets
from tyssue.core.monolayer import Monolayer

In [19]:
dsets = load_datasets('../data/hf5/small_ellipsoid.hf5',
                      data_names=['vert', 'edge', 'face', 'cell'])
specs = config.geometry.bulk_spec()

mono = Monolayer('mono', datasets=dsets, specs=specs)

In [20]:
colors = pd.DataFrame(np.zeros((mono.Nv, 3)),
                      index=mono.vert_df.index,
                      columns=['R', 'G', 'B'])

apical_color = [0.3, 0.6, 0.3]
basal_color = [0.6, 0.3, 0.3]

for i, c in enumerate('RGB'):
    colors.loc[mono.apical_verts, c] = apical_color[i]
    colors.loc[mono.basal_verts, c] = basal_color[i]
draw_specs['vert']['color'] = colors
draw_specs['edge']['width'] = 1.5

In [21]:
renderer = view_3js(mono, **draw_specs)


display(renderer)

In [22]:
import pythreejs as tjs

In [81]:
apical_sheet = mono.get_sub_sheet('apical')
apical_sheet.reset_index()
apical_sheet.reset_topo()
from tyssue.core.objects import ordered_vert_idxs
from matplotlib.colors import rgb2hex

vertices, faces, _ = apical_sheet.vertex_mesh(['x', 'y', 'z'])
geometry = tjs.FaceGeometry(vertices=list(vertices.flat),
                            facen=[list(f) for f in faces],
                            colors=face_colors)
material=tjs.LambertMaterial(color='green', side='FrontSide')
material.transparent = True
material.opacity = 0.3
surf = tjs.Mesh(geometry=geometry, material=material)


scene = tjs.Scene(children=[surf,
                            tjs.DirectionalLight(color='#ffaabb',
                                                 position=[0, 5, 0]),
                            tjs.AmbientLight(color='#cccccc')])
c = tjs.PerspectiveCamera(position=[5, 5, 3], up=[0, 0, 1],
                      children=[tjs.DirectionalLight(color='red',
                                                     position=[3, 5, 1],
                                                     intensity=0.6)])
renderer = tjs.Renderer(camera=c, scene=scene, 
                        controls=[tjs.OrbitControls(controlling=c)])
display(renderer)

NameError: name 'tjs' is not defined

> <ipython-input-81-5a451fb1f0a3>(8)<module>()
      6 
      7 vertices, faces, _ = apical_sheet.vertex_mesh(['x', 'y', 'z'])
----> 8 geometry = tjs.FaceGeometry(vertices=list(vertices.flat),
      9                             facen=[list(f) for f in faces],
     10                             colors=face_colors)

ipdb> c


In [31]:
scene.children